In [13]:
import requests
from bs4 import BeautifulSoup
import os
import urllib
import time
import datetime
import json
import csv
from tqdm.notebook import tqdm
import pandas as pd

In [14]:
# 填入供應商
# 範例: 真愛密碼 -> JCode
vendor = 'JCode'

In [15]:
# 填入"商品系列"網址第一頁
URL = "https://www.jcode.com.tw/categories/baishatun?page=1&sort_by=&order_by=&limit=24"

In [16]:
# 填入“商品系列”縮寫以便命名
#範例: 2024白沙屯媽祖 -> BST2024
type = 'BST2024'

In [17]:
# 填入輸出檔名(預設為廠商名+商品系列)
filename = vendor + type + '.csv'

In [18]:
# 以下為csv表格中必填資料(不常改動)

# 自訂"Product type"
sku = ''
# 是否上架Shopify賣場: "TRUE" 或 "FALSE"
published = "TRUE"
# 是否有子類別(尺寸、顏色、材質等等)，通常設為"無"
o1n = "Title"
o1v = "Default Title"
o2n, o2v, o3n, o3v = '', '', '', ''
# 重量 Variant Grams
vargram = 0
# Variant Inventory Tracker, Variant Inventory Qty, Variant Inventory Policy, Variant Fulfillment Service
vit, viq, vip, vfs = 'shopify', 1, 'deny', 'manual'
# 價格 Variant Price, Variant Compare At Price
v_price = 0
vcap = 0
# 運送 Variant Requires Shipping
vrship = 'TRUE'
# 稅 Variant Taxable
vtax = 'TRUE'
# 條碼 Variant Barcode
vbar = ''
# 圖片地址,順序 Image Src, Image Position, Image Alt Text
imgalt = ''
# 禮品卡 Gift Card
giftcard = 'FALSE'
# SEO Title, SEO Description
SEOtitle, SEOdis = '', ''
# Google Shopping / Google Product Category, Google Shopping / Gender,
# Google Shopping / Age Group, Google Shopping / MPN, Google Shopping / Condition,
# Google Shopping / Custom Product, Google Shopping / Custom Label 0,
# Google Shopping / Custom Label 1, Google Shopping / Custom Label 2,
# Google Shopping / Custom Label 3, Google Shopping / Custom Label 4
g1, g2 , g3, g4, g5, g6, g7, g8, g9, g10, g11 = '', '', '', '', '', '', '', '', '', '', ''
# Variant Image, Variant Weight Unit, Variant Tax Code, Cost per item
vimg, vwu, vtc, cpi = '', '', '', ''
# Included / 台灣, Price / 台灣, Compare At Price / 台灣, 
# Included / 國際, Price / 國際, Compare At Price / 國際
TWinc, TWp, TWcap, INTinc, INTp, INTcap = 'TRUE', '', '', 'TRUE', '', ''
# Status
status = 'archived'

In [19]:
# 自動找出該系列所有商品網址
page_num = 1
url_list = []

while True:
    # 自動換頁
    if page_num < 10:
        URL = URL[:URL.find('page')+5] + str(page_num) + URL[URL.find('page')+6:]
    else:
        URL = URL[:URL.find('page')+5] + str(page_num) + URL[URL.find('page')+7:]

    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, 'html.parser')

    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    # 抓出該頁所有產品網址
    scrap1 = soup2.find_all(class_="Product-item --plp-product-custom-label-rounded-rectangle")

    for i in scrap1:
        url_list.append(i.get('href'))

    page_num += 1

    if scrap1 == []:
        break

print(url_list)

['https://www.jcode.com.tw/products/白沙屯媽祖-發財金-黃金紅包袋-1', 'https://www.jcode.com.tw/products/白沙屯媽祖160週年紀念-黃金墜飾-1', 'https://www.jcode.com.tw/products/白沙屯媽祖-發財金---黃金擺飾--擺件', 'https://www.jcode.com.tw/products/白沙屯-媽祖香火袋-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-媽祖-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-粉紅超跑神轎---黃金擺飾--擺件', 'https://www.jcode.com.tw/products/白沙屯-招財獅公母-黃金擺件', 'https://www.jcode.com.tw/products/白沙屯-上上籤---黃金擺飾--擺件', 'https://www.jcode.com.tw/products/白沙屯-媽祖招財古錢-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-保庇平安媽祖-編織金手鍊', 'https://www.jcode.com.tw/products/白沙屯-媽祖-迷你-編織金手鍊', 'https://www.jcode.com.tw/products/白沙屯-一生好命媽祖-編織金手鍊', 'https://www.jcode.com.tw/products/白沙屯-平安蘋果媽祖-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-粉紅超跑神轎-迷你-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-平安蘋果媽祖-編織金手鍊', 'https://www.jcode.com.tw/products/白沙屯-媽祖招財古錢-編織金手鍊', 'https://www.jcode.com.tw/products/白沙屯-娘傘媽祖-黃金墜子', 'https://www.jcode.com.tw/products/白沙屯-旺財媽祖-黃金墜子', 'https://www.jcode.com.tw/pr

In [20]:
csv_header = ['Handle', 'Title', 'Body (HTML)', 'Vendor', 'Product Category',
               'Type', 'Tags', 'Published', 'Option1 Name', 'Option1 Value', 
               'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value', 
               'Variant SKU', 'Variant Grams', 'Variant Inventory Tracker', 
               'Variant Inventory Qty', 'Variant Inventory Policy',  
               'Variant Fulfillment Service', 'Variant Price', 
               'Variant Compare At Price', 'Variant Requires Shipping', 'Variant Taxable', 
               'Variant Barcode', 'Image Src', 'Image Position', 'Image Alt Text', 
               'Gift Card', 'SEO Title', 'SEO Description', 
               'Google Shopping / Google Product Category', 
               'Google Shopping / Gender', 'Google Shopping / Age Group', 
               'Google Shopping / MPN', 'Google Shopping / Condition', 
               'Google Shopping / Custom Product', 'Google Shopping / Custom Label 0', 
               'Google Shopping / Custom Label 1', 'Google Shopping / Custom Label 2', 
               'Google Shopping / Custom Label 3', 'Google Shopping / Custom Label 4',
               'Variant Image', 'Variant Weight Unit', 'Variant Tax Code', 
               'Cost per item', 'Included / 台灣', 'Price / 台灣', 'Compare At Price / 台灣', 
               'Included / 國際', 'Price / 國際', 'Compare At Price / 國際', 'Status']
df = pd.DataFrame(columns=csv_header)

In [21]:
# 將所有網址的商品資料抓齊
for url in tqdm(url_list):
    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}

    page = requests.get(url, headers=headers)

    soup1 = BeautifulSoup(page.content, 'html.parser')

    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    # 抓取需要的商品的文字資訊
    title = soup2.find(class_='Product-title').text.strip()

    title = title.replace("/", "_")

    description = soup2.find(class_='Product-summary Product-summary-block').text.strip()

    # 自動歸類"Product Category"
    prod_cat = ""
    if '手鍊' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Bracelets"
    if '戒指' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Rings"
    if '項鍊' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Necklaces"
    if '耳環' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Earrings"
    if '墜飾' in title  or '墜子' in title :
        prod_cat = "Apparel & Accessories > Jewelry > Charms & Pendants"
    if '擺件' in title or '擺飾' in title or '禮盒' in title:
        prod_cat = "Apparel & Accessories > Jewelry > Jewelry Sets"

    # 自動歸類“Tags”
    tag = []
    if '金' in title:
        tag.append('黃金')
    if '銀' in title:
        tag.append('銀')
    if '手鍊' in title:
        tag.append('手鍊')
    if '戒指' in title:
        tag.append('戒指')
    if '項鍊' in title:
        tag.append('項鍊')
    if '耳環' in title:
        tag.append('耳環')
    if '墜飾' in title  or '墜子' in title :
        tag.append('墜子')
    if '擺件' in title or '擺飾' in title or '禮盒' in title:
        tag.append('擺件')
    tag = ", ".join(tag)

    # 抓取商品圖片網址
    image = soup2.find(type='application/ld+json').text.strip()

    #image = image.replace("'", "\"")

    image = json.loads(image)['image']

    # 將資料寫入表格中
    data = []
    for index, url in enumerate(image):
        if index == 0:
            data = [[title, title, description, vendor, prod_cat, type, tag, published, o1n, o1v, 
                o2n, o2v, o3n, o3v, sku, vargram, vit, viq, vip, vfs, v_price, vcap, vrship, vtax, 
                vbar, url, index+1, imgalt, giftcard, SEOtitle, SEOdis, g1 , g2, g3, g4, g5, 
                g6, g7, g8, g9, g10, g11, vimg, vwu, vtc, cpi, TWinc, TWp, TWcap, INTinc, INTp, 
                INTcap, status]]
        else:
            data.append([title, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
                        '', '', '', '', '', '', '', url, index+1, '', '', '', '', '', '', '', '', 
                        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''])
    
    new_df = pd.DataFrame(data, columns=csv_header)
    df = pd.concat([df, new_df], ignore_index=True)

    # 設置延遲時間
    time.sleep(0.3)

with open(filename, 'w', newline='', encoding='UTF8') as file:
    df.to_csv(file, index=False)

  0%|          | 0/57 [00:00<?, ?it/s]

In [22]:
# 商品數量、圖片總量
print(len(df['Title'].unique())-1, len(df['Handle']))

57 194
